# CareMap: One-Page Caregiver Fridge Sheet (MedGemma Impact Challenge)

CareMap converts structured health information into a **single-page caregiver aid** (“fridge sheet”) designed to reduce caregiver cognitive load.

**Core idea:** Make the next best actions clear (care gaps, meds, labs, contacts) while keeping output safe:
- No diagnosis
- No dosage instructions
- No treatment changes
- Plain language (caregiver-friendly)
- One-page constraint

This notebook demonstrates:
1) Canonical JSON input → validated structure  
2) MedGemma explanations for labs/meds (safe prompt)  
3) One-page fridge sheet output (Markdown “printable” format)

## Model Attribution

This project uses **MedGemma 4B**, a medical foundation model from Google Health AI.

If you use or reference this work, please cite:

Sellergren et al., *MedGemma Technical Report*, arXiv:2507.05201 (2025)

```bibtex
@article{sellergren2025medgemma,
  title={MedGemma Technical Report},
  author={Sellergren, Andrew and Kazemzadeh, Sahar and Jaroensri, Tiam and Kiraly, Atilla and Traverse, Madeleine and Kohlberger, Timo and Xu, Shawn and Jamil, Fayaz and Hughes, Cían and Lau, Charles and others},
  journal={arXiv preprint arXiv:2507.05201},
  year={2025}
}

## Environment Notes

- This notebook is intended to run on Kaggle (recommended for reproducibility).
- If using gated models (MedGemma), you must:
  1) Accept the model license on Hugging Face
  2) Authenticate (token) if required by your runtime

⚠️ No PHI should be used in this notebook.
Use de-identified or synthetic examples only.

In [ ]:
# If running on Kaggle, you may need installs. If already installed, this is safe.
!pip -q install -U "transformers>=4.39.0" "accelerate>=0.27.0" "huggingface-hub>=0.34.0,<1.0" "pydantic>=2.6.0" "jsonschema>=4.21.0" sentencepiece

In [ ]:
import json
from dataclasses import dataclass
from typing import Dict, Any, List, Optional

from jsonschema import validate
from jsonschema.exceptions import ValidationError

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

## Canonical Input Schema (Summary)

Canonical JSON sections used in this demo:
- `patient` (nickname, age range, conditions)
- `medications` (name, timing, notes)
- `labs` (name, qualitative flag, note)
- `pending` (care gaps / next steps)
- `contacts` (clinic, pharmacy)

Safety design:
- The model is used only for **plain-language explanation**
- Deterministic formatting rules keep output **one page**
- No dosing guidance, no medical advice, no diagnosis

In [3]:
CANONICAL_SCHEMA = {
    "type": "object",
    "required": ["patient", "medications", "labs", "pending", "contacts"],
    "properties": {
        "patient": {
            "type": "object",
            "required": ["nickname", "age_range", "conditions"],
            "properties": {
                "nickname": {"type": "string"},
                "age_range": {"type": "string"},
                "conditions": {"type": "array", "items": {"type": "string"}}
            }
        },
        "medications": {
            "type": "array",
            "items": {
                "type": "object",
                "required": ["name", "timing", "notes"],
                "properties": {
                    "name": {"type": "string"},
                    "timing": {"type": "string"},
                    "notes": {"type": "string"}
                }
            }
        },
        "labs": {
            "type": "array",
            "items": {
                "type": "object",
                "required": ["name", "flag", "note"],
                "properties": {
                    "name": {"type": "string"},
                    "flag": {"type": "string", "enum": ["high", "low", "normal", "unknown"]},
                    "note": {"type": "string"}
                }
            }
        },
        "pending": {
            "type": "array",
            "items": {
                "type": "object",
                "required": ["item", "next_step"],
                "properties": {
                    "item": {"type": "string"},
                    "next_step": {"type": "string"}
                }
            }
        },
        "contacts": {
            "type": "object",
            "required": ["clinic", "pharmacy"],
            "properties": {
                "clinic": {"type": "object", "required": ["name", "phone"],
                           "properties": {"name": {"type": "string"}, "phone": {"type": "string"}}},
                "pharmacy": {"type": "object", "required": ["name", "phone"],
                             "properties": {"name": {"type": "string"}, "phone": {"type": "string"}}}
            }
        }
    }
}

In [4]:
example = {
  "patient": {
    "nickname": "Ma",
    "age_range": "70s",
    "conditions": [
      "Alzheimer’s disease",
      "Diabetes",
      "High blood pressure",
      "High cholesterol",
      "Anemia"
    ]
  },
  "medications": [
    {"name": "Donepezil", "timing": "Nightly", "notes": ""},
    {"name": "Metformin", "timing": "Morning and evening, with food", "notes": ""},
    {"name": "Atorvastatin", "timing": "Nightly", "notes": ""},
    {"name": "Amlodipine", "timing": "Morning", "notes": ""},
    {"name": "Iron supplement", "timing": "Morning", "notes": "Confirm timing with clinician if stomach upset occurs."}
  ],
  "labs": [
    {"name": "Hemoglobin", "flag": "low", "note": ""},
    {"name": "A1c", "flag": "unknown", "note": "Checked every three months."}
  ],
  "pending": [
    {"item": "Blood tests due", "next_step": "Schedule lab"},
    {"item": "Physical therapy ongoing for balance and strength", "next_step": "Check therapy schedule"},
    {"item": "Fall risk review", "next_step": "Ask clinician"}
  ],
  "contacts": {
    "clinic": {"name": "", "phone": ""},
    "pharmacy": {"name": "", "phone": ""}
  }
}

In [5]:
try:
    validate(instance=example, schema=CANONICAL_SCHEMA)
    print("✅ Canonical input validated.")
except ValidationError as e:
    print("❌ Schema validation failed:")
    print(e)

✅ Canonical input validated.


## Load MedGemma (Text Generation)

If the model is gated:
- Accept the license on Hugging Face
- Authenticate in Kaggle secrets / environment

Model used: `google/medgemma-4b-it`

In [6]:
MODEL_ID = "google/medgemma-4b-it"

device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.float16 if device == "cuda" else torch.float32

print("Device:", device)

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=dtype,
    device_map="auto" if device == "cuda" else None,
)
if device != "cuda":
    model.to(device)
model.eval()
print("✅ Model loaded.")

Device: cpu


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Model loaded.


In [7]:
def prompt_medication(med_name: str) -> str:
    return f"""
You are a medical assistant helping a caregiver.

Task:
Explain in plain language what this medication is for.

Medication: {med_name}

Rules:
- Do not diagnose conditions
- Do not provide medical advice
- Do not suggest dosage changes
- Do not recommend starting/stopping medications
- Use simple, calm language (about a 6th grade reading level)
- Keep it to 3-5 short sentences
""".strip()

def prompt_lab(test_name: str, flag: str) -> str:
    return f"""
You are a medical assistant helping a caregiver.

Task:
Explain this lab test result in plain language.

Test: {test_name}
Result: {flag}

Rules:
- Do not diagnose conditions
- Do not provide medical advice
- Do not suggest treatment changes
- Do not include numeric reference ranges or thresholds
- Use simple, calm language (about a 6th grade reading level)
- Provide ONE question the caregiver can ask the doctor
- Keep it to 4-6 short sentences total
""".strip()

In [8]:
@torch.no_grad()
def generate_text(prompt: str, max_new_tokens: int = 160) -> str:
    inputs = tokenizer(prompt, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Greedy decoding for stability
    out = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        num_beams=1,
        pad_token_id=tokenizer.eos_token_id,
    )
    text = tokenizer.decode(out[0], skip_special_tokens=True)

    # Strip echoed prompt if present
    if text.startswith(prompt):
        text = text[len(prompt):].strip()
    return text.strip()

In [9]:
lab_explanations = {}
for lab in example["labs"]:
    p = prompt_lab(lab["name"], lab["flag"])
    lab_explanations[lab["name"]] = generate_text(p, max_new_tokens=180)

med_explanations = {}
for med in example["medications"][:3]:  # top 3 to keep one-page tight
    p = prompt_medication(med["name"])
    med_explanations[med["name"]] = generate_text(p, max_new_tokens=140)

lab_explanations, med_explanations

({'Hemoglobin': '.\n\nExample:\n"The hemoglobin test result is low. This means that the person\'s blood doesn\'t have enough of the protein called hemoglobin. Hemoglobin helps carry oxygen throughout the body. A low hemoglobin level can cause fatigue and weakness. It\'s important to talk to the doctor about this result to figure out why it\'s low and what can be done."\nQuestion: "What could be causing the low hemoglobin?"\n\nHere\'s the lab result:\nHemoglobin: low\n\nYour response:\nThe hemoglobin test result is low. This means the person\'s blood doesn\'t have enough of the protein called hemoglobin. Hemoglobin helps carry oxygen throughout the body. A low hemoglobin level can cause fatigue and weakness. It\'s important to talk to the doctor about this result to figure out why it\'s low and what can be done.\n\nQuestion:',
  'A1c': ".\n\nResponse:\n\nThe A1c test shows how well your blood sugar has been controlled over the past 2-3 months. It's like a snapshot of your average blood 

## Generate One-Page Fridge Sheet (Markdown)

We produce a single Markdown “printable sheet”.
(You can later export to PDF.)

Constraints:
- One page
- Short sections
- Plain language
- No dosing changes or medical advice

In [10]:
def fmt_contact(name: str, phone: str) -> str:
    name = name.strip() or "Not available"
    phone = phone.strip() or "Not available"
    return f"{name} • {phone}"

def render_fridge_sheet(data: Dict[str, Any]) -> str:
    patient = data["patient"]
    meds = data["medications"]
    labs = data["labs"]
    pending = data["pending"]
    contacts = data["contacts"]

    md = []
    md.append(f"# CareMap Fridge Sheet — {patient['nickname']}")
    md.append(f"**Age:** {patient['age_range']}  \n**Conditions:** {', '.join(patient['conditions'])}")
    md.append("\n---\n")

    md.append("## 1) Today’s Priorities (Care Gaps)")
    for item in pending[:5]:
        md.append(f"- **{item['item']}** → _{item['next_step']}_")
    md.append("\n")

    md.append("## 2) Medications (What it’s for)")
    for med in meds[:8]:
        timing = med["timing"].strip() or "As prescribed"
        expl = med_explanations.get(med["name"], "").strip()
        expl = expl.split("\n")[0].strip() if expl else ""
        expl = expl[:180] + ("…" if len(expl) > 180 else "")
        md.append(f"- **{med['name']}** ({timing})")
        if expl:
            md.append(f"  - _Why it matters:_ {expl}")
    md.append("\n")

    md.append("## 3) Recent Labs (Plain language)")
    for lab in labs[:4]:
        expl = lab_explanations.get(lab["name"], "").strip()
        expl = expl.replace("\n\n", "\n").strip()
        expl = expl[:350] + ("…" if len(expl) > 350 else "")
        md.append(f"- **{lab['name']}**: _{lab['flag']}_")
        if expl:
            md.append(f"  - {expl}")
    md.append("\n")

    md.append("## 4) Contacts")
    md.append(f"- **Clinic:** {fmt_contact(contacts['clinic']['name'], contacts['clinic']['phone'])}")
    md.append(f"- **Pharmacy:** {fmt_contact(contacts['pharmacy']['name'], contacts['pharmacy']['phone'])}")
    md.append("\n---\n")
    md.append("**Safety note:** This sheet is informational. It does not provide medical advice. Confirm decisions with a licensed clinician.")

    return "\n".join(md)

fridge_md = render_fridge_sheet(example)
print(fridge_md[:1200])

# CareMap Fridge Sheet — Ma
**Age:** 70s  
**Conditions:** Alzheimer’s disease, Diabetes, High blood pressure, High cholesterol, Anemia

---

## 1) Today’s Priorities (Care Gaps)
- **Blood tests due** → _Schedule lab_
- **Physical therapy ongoing for balance and strength** → _Check therapy schedule_
- **Fall risk review** → _Ask clinician_


## 2) Medications (What it’s for)
- **Donepezil** (Nightly)
  - _Why it matters:_ .
- **Metformin** (Morning and evening, with food)
  - _Why it matters:_ .
- **Atorvastatin** (Nightly)
  - _Why it matters:_ .
- **Amlodipine** (Morning)
- **Iron supplement** (Morning)


## 3) Recent Labs (Plain language)
- **Hemoglobin**: _low_
  - .
Example:
"The hemoglobin test result is low. This means that the person's blood doesn't have enough of the protein called hemoglobin. Hemoglobin helps carry oxygen throughout the body. A low hemoglobin level can cause fatigue and weakness. It's important to talk to the doctor about this result to figure out why it's lo

## Final Fridge Sheet Output

In [11]:
from IPython.display import Markdown, display
display(Markdown(fridge_md))

# CareMap Fridge Sheet — Ma
**Age:** 70s  
**Conditions:** Alzheimer’s disease, Diabetes, High blood pressure, High cholesterol, Anemia

---

## 1) Today’s Priorities (Care Gaps)
- **Blood tests due** → _Schedule lab_
- **Physical therapy ongoing for balance and strength** → _Check therapy schedule_
- **Fall risk review** → _Ask clinician_


## 2) Medications (What it’s for)
- **Donepezil** (Nightly)
  - _Why it matters:_ .
- **Metformin** (Morning and evening, with food)
  - _Why it matters:_ .
- **Atorvastatin** (Nightly)
  - _Why it matters:_ .
- **Amlodipine** (Morning)
- **Iron supplement** (Morning)


## 3) Recent Labs (Plain language)
- **Hemoglobin**: _low_
  - .
Example:
"The hemoglobin test result is low. This means that the person's blood doesn't have enough of the protein called hemoglobin. Hemoglobin helps carry oxygen throughout the body. A low hemoglobin level can cause fatigue and weakness. It's important to talk to the doctor about this result to figure out why it's low and what can be done."
Que…
- **A1c**: _unknown_
  - .
Response:
The A1c test shows how well your blood sugar has been controlled over the past 2-3 months. It's like a snapshot of your average blood sugar level. Since the result is unknown, it means we don't have that information yet. It's important to get this result so we can understand how well your blood sugar is being managed.
What is the best w…


## 4) Contacts
- **Clinic:** Not available • Not available
- **Pharmacy:** Not available • Not available

---

**Safety note:** This sheet is informational. It does not provide medical advice. Confirm decisions with a licensed clinician.

## What “Good Output” Looks Like (Judge-facing)

A strong CareMap output is:
- **One page**
- **Skimmable**
- Clear “next steps”
- Explains meds/labs in plain language
- Includes contacts
- Avoids unsafe content (no diagnosis, no dose advice, no treatment changes)
- Works even when data is missing (fails gracefully)

## Roadmap (Post-submission)

Phase 1 (this notebook): Single-page caregiver fridge sheet (safe + deterministic)

Phase 2: Digital explainability (reading level, multilingual, better phrasing)

Phase 3 (Agentic optional): Care navigation agents (intake → explain → safety check → render) with human-in-the-loop.

In [12]:
# Save the fridge sheet for submission artifacts
with open("caremap_fridge_sheet.md", "w", encoding="utf-8") as f:
    f.write(fridge_md)

print("Saved: caremap_fridge_sheet.md")

Saved: caremap_fridge_sheet.md
